<br>
<u>Notebook Nine</u> | 
<a href=https://leone.gdn/DLL target=_blank>Report</a> | 
<a href=https://github.com/andrealeone/DLL>Repository</a>
<br><br>
<b>Task 1: Attributes computation</b><br><br>
Andrea Leone, Alessandro E. Piotti<br>
Deep Learning Lab — University of Trento<br>
December 2021
<hr><br><br>

In [1]:
import project 

import os
import pandas          as pd
import pickle

import torch
import torchvision
import networks

from tqdm.notebook import tqdm

project.notebook()

<hr><br><br>

## Attributes processing

<br><br>Load train, test and query images

In [2]:
images     =  dict() 
images_pkl = 'images.pkl'
sets       = [ 'train', 'test', 'queries' ] 
irm        = torchvision.io.ImageReadMode.RGB 

if images_pkl in os.listdir('./data/'):
    
    with open('./data/' + images_pkl, 'rb') as file: 
        images = pickle.load( file )

else:
    
    for key in sets: 

        path         = '../dataset/{}/'.format(key)
        paths        = sorted([(path + file) for file in os.listdir(path)])
        images[key]  = dict()

        for image_file in tqdm(paths):

            image_id = image_file.split('/')[-1].split('.')[0]
            tensor   = torchvision.io.read_image( image_file,irm )
            tensor   = tensor.unsqueeze(0).float()

            images[key][image_id] = tensor
    
    with open('./data/' + images_pkl, 'wb') as file: 
        pickle.dump( images, file )

<br><br>Load the model sources

In [3]:
models  = pickle.load( open('./data/models.v2.pkl', 'rb') ) 

<br><br>Init the super-model

In [4]:
model   = lambda _input : project.super_model(_input, src=models) 

<br><br>Test the output

In [6]:
print( model( images['test']['000000'] ).int().detach().numpy() + 1 ) 

[2 1 2 1 1 2 2 1 1 2 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1]


<br><br>Compute the attribute vectors from both test and query images

In [ ]:
attributes = dict() 

for key in sets:
    
    attributes[key] = dict()
    tensors         = list( images[key].items() )
    
    for image_id, tensor in tqdm(tensors):
        
        attributes[key][image_id] = model( tensor ) + 1

<br><br>Export the results

In [ ]:
v  = '2'                        # release version 

with open('./data/attributes.v{}.pkl'.format(v), 'wb') as file: 
    pickle.dump(attributes, file)

for key in sets:
    
    pd.DataFrame( 
        data    = attributes[key].values(),
        index   = attributes[key].keys(),
        columns = [
            'age', 'backpack', 'bag', 'handbag',
            'clothes', 'down', 'up', 'hair', 'hat', 'gender',
            'upblack', 'upwhite', 'upred', 'uppurple',
            'upyellow', 'upgray', 'upblue', 'upgreen',
            'downblack', 'downwhite', 'downpink', 'downpurple', 'downyellow',
            'downgray', 'downblue', 'downgreen', 'downbrown'
        ]
    ).to_csv('./data/attributes.{}.v{}.csv'.format(key, v))

<br><br>